In [0]:
import torch
import torchvision
from google.colab import drive
import os

In [2]:
torchvision.models.resnet50(pretrained=True)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
resnet50 = torchvision.models.resnet50(pretrained=True)

In [4]:
resnet50.fc

Linear(in_features=2048, out_features=1000, bias=True)

In [0]:
resnet50.fc = torch.nn.Linear(2048, 5, bias=True)

# 데이터 로드

In [6]:
drive.mount('/drive/')

Drive already mounted at /drive/; to attempt to forcibly remount, call drive.mount("/drive/", force_remount=True).


In [0]:
data_dir = '/drive/My Drive/빅데이터 청년인재 고려대 과정 1조/K-Data 고려대학교 빅데이터 청년인재 교육과정 1조/프로젝트/씨앗/A'

In [8]:
# path 지정함
# os.path.join으로 path 합치는 거 해봄.

data_path = {x:os.path.join(data_dir, x) for x in ["train", "test"]}
data_path

{'test': '/drive/My Drive/빅데이터 청년인재 고려대 과정 1조/K-Data 고려대학교 빅데이터 청년인재 교육과정 1조/프로젝트/씨앗/A/test',
 'train': '/drive/My Drive/빅데이터 청년인재 고려대 과정 1조/K-Data 고려대학교 빅데이터 청년인재 교육과정 1조/프로젝트/씨앗/A/train'}

In [0]:
# Compose는 조립한다는 뜻이다.
# transforms의 함수들을 리스트로 만들어서 인자로 전달하면, pipeline같은 역할을 한다.

# data augmentation도 여기서 처리할 수 있다.
# 하지만 일단 처음 돌리는거이므로 pass.

data_transforms = torchvision.transforms.Compose([
    torchvision.transforms.CenterCrop(2688), # 중앙을 중심을 크롭해준다.
                                 # 인자가 int면 정방형, 가로세로 정하려면 시퀀스로.
    torchvision.transforms.Resize(224),
    torchvision.transforms.ToTensor(), # tensor 객체로 바꿔준다. 
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.3, 0.3, 0.3)),# 노말라이즈한다.
                                                           # 평균과 분산을 넣는다. 채널 수 만큼 넣는다.
                                                           # 일단 아무 숫자나 넣어본다.
])

In [29]:
# ImageFolder 함수는 path에서 이미지를 load해온다. 
# ToTensor하기 전에는 PILimage로 가져온다. (이름 정확히 기억 안남)

stem_datasets = {x : torchvision.datasets.ImageFolder(data_path[x], data_transforms) \
              for x in ["train", 'test']}
stem_datasets

{'test': Dataset ImageFolder
     Number of datapoints: 198
     Root location: /drive/My Drive/빅데이터 청년인재 고려대 과정 1조/K-Data 고려대학교 빅데이터 청년인재 교육과정 1조/프로젝트/씨앗/A/test,
 'train': Dataset ImageFolder
     Number of datapoints: 800
     Root location: /drive/My Drive/빅데이터 청년인재 고려대 과정 1조/K-Data 고려대학교 빅데이터 청년인재 교육과정 1조/프로젝트/씨앗/A/train}

In [30]:
# dataset을 딥러닝 학습할 데이터 셋으로 만들어 준다.... 
# batch_size를 지정할 수 있다.
# shuffle을 True로 하는게 좋다. batch돌면서 랜덤하게 데이터를 준다.
# num_workers가 0이면 모든 코어를 다 쓴다?는거 같음.

# DataLoader한 후에 train, test를 나눌 수 없다. train,test를 split한 다음에 DataLoader로 넘겨줘야한다.

dataloaders = {x: torch.utils.data.DataLoader(stem_datasets[x], batch_size=4, shuffle=True, num_workers=4) \
              for x in ['train', 'test']}
dataloaders

{'test': <torch.utils.data.dataloader.DataLoader at 0x7f5819cc2cf8>,
 'train': <torch.utils.data.dataloader.DataLoader at 0x7f5819cc2e80>}

In [12]:
stem_datasets['train'].classes

['결명자_train',
 '구기자_train',
 '나복자_train',
 '보골지_train',
 '우방자_train']

## 학습 함수

In [0]:
def train_network(net,optimizer,trainloader, epochs=5):
    for epoch in range(epochs):  # loop over the dataset multiple times
                               # epochs = 전체 데이터가 한바퀴 돈 것
        running_loss = 0.0 # running loss를 저장하기 위한 변수입니다. 
        for i, data in enumerate(trainloader): # 한 Epoch 만큼 돕니다. 매 iteration 마다 정해진 Batch size 만큼 데이터를 뱉습니다. 
            # get the inputs
            inputs, labels = data # DataLoader iterator의 반환 값은 input_data 와 labels의 튜플 형식입니다. 
            inputs = inputs.to(device) # Pytorch에서 nn.Module 에 넣어 Backprop을 계산 하기 위해서는 gpu 연동을 이와 같이 해줘야 합니다.
            labels = labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()    #  현재 기존의 backprop을 계산하기 위해서 저장했던 activation buffer 를 비웁니다.
            # forward + backward + optimize
            outputs = net(inputs) # input 을 넣은 위 network 로 부터 output 을 얻어냅니다. 
            loss = criterion(outputs, labels) # loss fucntion에 주어진 target과 output 의 score를 계산하여 반환합니다. 
            loss.backward() # * Scalar Loss value를 Backward() 해주게 되면 주어진 loss값을 바탕으로 backpropagation이 진행됩니다. 
            optimizer.step() # 계산된 Backprop 을 바탕으로 optimizer가 gradient descenting 을 수행합니다. 

            # print statistics 
            running_loss += loss.item()
            if (i+1) % 10 == 0:    # print every 500 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

    print('Finished Training')

In [0]:
def test(model,test_loader):
    model.eval() # Eval Mode 왜 해야 할까요?  --> nn.Dropout BatchNorm 등의 Regularization 들이 test 모드로 들어가게 되기 때문입니다. 
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)  # 기존의 train function의 data 처리부분과 같습니다. 
        output = model(data) 
        pred = output.max(1, keepdim=True)[1] # get the index of the max 
        correct += pred.eq(target.view_as(pred)).sum().item() # 정답 데이터의 갯수를 반환합니다. 

    test_loss /= len(test_loader.dataset)
    print('\nTest set:  Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## 학습!

In [0]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# dummy = torchvision.models.vgg11_bn(pretrained=True).to(device)

In [0]:
def count_parameters(model): # 모델 파라미터 개수를 리턴하는 함수를 하나 만들어둡니다
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [19]:
lr = 0.001
resnet50 = resnet50.to(device)
# dummy = dummy.to(device)
criterion = torch.nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = torch.optim.Adam(resnet50.parameters(), lr=lr) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 
# optimizer = torch.optim.Adam(dummy.parameters(), lr=lr)

print('[info] number of model parameter - %d'%(count_parameters(resnet50))) # 방금 구성한 모형의 파라미터 개수를 프린트 해봅니다.

[info] number of model parameter - 23518277


In [20]:
train_network(resnet50,optimizer,dataloaders["train"], epochs=5)

[1,    10] loss: 0.208
[1,    20] loss: 0.125
[1,    30] loss: 0.111
[1,    40] loss: 0.109
[1,    50] loss: 0.122
[1,    60] loss: 0.081
[1,    70] loss: 0.127
[1,    80] loss: 0.140
[1,    90] loss: 0.102
[1,   100] loss: 0.127
[1,   110] loss: 0.111
[1,   120] loss: 0.092
[1,   130] loss: 0.116
[1,   140] loss: 0.091
[1,   150] loss: 0.098
[1,   160] loss: 0.080
[1,   170] loss: 0.077
[1,   180] loss: 0.098
[1,   190] loss: 0.076
[1,   200] loss: 0.071
[2,    10] loss: 0.090
[2,    20] loss: 0.070
[2,    30] loss: 0.092
[2,    40] loss: 0.073
[2,    50] loss: 0.084
[2,    60] loss: 0.088
[2,    70] loss: 0.072
[2,    80] loss: 0.050
[2,    90] loss: 0.086
[2,   100] loss: 0.075
[2,   110] loss: 0.079
[2,   120] loss: 0.075
[2,   130] loss: 0.052
[2,   140] loss: 0.056
[2,   150] loss: 0.080
[2,   160] loss: 0.085
[2,   170] loss: 0.082
[2,   180] loss: 0.044
[2,   190] loss: 0.064
[2,   200] loss: 0.087
[3,    10] loss: 0.065
[3,    20] loss: 0.092
[3,    30] loss: 0.076
[3,    40] 

In [31]:
test(resnet50, dataloaders["test"])

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f586b2177f0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f586b2177f0>>
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f586b2177f0>>
Traceback (most recent call last):
    w.join()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 677, in __del__
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    self._shutdown_workers()
    assert self._parent_pid == o


Test set:  Accuracy: 179/198 (90%)



In [0]:
torch.save(resnet50.state_dict, '/drive/My Drive/Colab Notebooks/resnet50_5epochs.pth')

In [22]:
data_dir

'/drive/My Drive/빅데이터 청년인재 고려대 과정 1조/K-Data 고려대학교 빅데이터 청년인재 교육과정 1조/프로젝트/씨앗/A'